# Setup

In [3]:
import Pkg

In [5]:
Pkg.activate(".")

  Activating new environment at `~/workspace/repo/code/SpinGlassExhaustive.jl/notebooks/Project.toml`


In [8]:
Pkg.add("BenchmarkTools")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
    Updating `~/workspace/repo/code/SpinGlassExhaustive.jl/notebooks/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
    Updating `~/workspace/repo/code/SpinGlassExhaustive.jl/notebooks/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
  [682c06a0] + JSON v0.21.2
  [69de0a69] + Parsers v2.2.1
  [ade2ca70] + Dates
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [a63ad114] + Mmap
  [de0858da] + Printf
  [9abbd945] + Profile
  [9a3f8284] + Random
  [ea8e919c] + SHA
  [9e88b42a] + Serialization
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
  [cf7118a7] + UUIDs
  [4ec0a83e] + Unicode
Precompiling project...
  ✓ BenchmarkTools
  1 dependency successfully precompiled in 2 seconds (2 already precompiled)


In [9]:
using CUDA
# using Distributions
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1342


In [10]:
Pkg.rm("CuQuantumInformation")

LoadError: The following package names could not be resolved:
 * CuQuantumInformation (not found in project or manifest)


# Create graph

In [8]:
N = 24

# graph = zeros(2^N)
graph = rand(Uniform(-5,5),N, N)
graph = graph * graph'

qubo = zeros(N,N)
constant = 0

for i in 1:N
    qubo[i,i] = 2 * graph[i,i]
    constant += graph[i,i]
    
    for j in 1:N
        if i!=j
            low, high = sort!([i, j])
            constant -= graph[low,high]*0.5
            qubo[i,i] -= 2*graph[low,high]
            qubo[low,high] += graph[low,high]*2
        end
    end
end


cu_graph = graph |> cu 
cu_qubo = qubo |> cu 

24×24 CuArray{Float32, 2}:
 480.931  -277.932    90.1892  374.921   …    70.8954   -29.1377   128.9
   0.0     183.769  -355.961   -88.6602      492.883    114.909    323.342
   0.0       0.0     360.272   170.883      -296.491   -221.944    215.987
   0.0       0.0       0.0     125.28         96.6045  -221.099     25.128
   0.0       0.0       0.0       0.0          78.4859    30.3461   146.944
   0.0       0.0       0.0       0.0     …   -84.8284   -19.9886  -192.794
   0.0       0.0       0.0       0.0         127.82    -195.864    169.458
   0.0       0.0       0.0       0.0         141.386    101.138    172.587
   0.0       0.0       0.0       0.0        -325.827     30.2404    42.7617
   0.0       0.0       0.0       0.0         171.242   -187.407   -191.93
   0.0       0.0       0.0       0.0     …  -115.604    166.026    101.976
   0.0       0.0       0.0       0.0        -254.741    239.954    345.444
   0.0       0.0       0.0       0.0        -254.628    -45.1084   -23.437


# Brute forece

In [9]:
function energy(graph, state_code)
    F = 0
    N = size(graph)[1]
    q = digits(state_code, base=2, pad=N) #|> reverse

    for i in 1:N
        F -= graph[i,i]*q[i]  
        for j in 1:N
            low, high = sort!([i, j])
            F -= graph[low,high]*q[i]*q[j]
        end
    end
    return F
end

@btime begin
    res = zeros(2^N)

    for k in 1:2^N
        F = energy(qubo, k)
        res[k] = F
    end

    sort!(res)
end

  556.675 s (9747561479 allocations: 869.62 GiB)


16777216-element Vector{Float64}:
 -24231.077370083785
 -24007.73227204339
 -23786.471185945367
 -23737.683247377267
 -23559.937229211268
 -23547.202474881113
 -23515.359843559236
 -23501.768257858384
 -23352.53134733795
 -23346.112190416345
 -23314.723981870287
 -23264.64242913477
 -23252.38476753919
      ⋮
    -80.06949943731726
    -75.29317379497036
    -54.98340679127875
    -34.369232716390655
    -24.935309810451827
      0.0
     68.14880961218113
    128.59712202064313
    205.54322704985094
    214.609155106332
    313.9736408190006
    368.9641657369085

# Brute-force GPU

In [10]:
function _energy(state_code, graph)
    F = 0
    N = size(graph)[1]
    
    sc1 = state_code    
    for i in 1:N
        qi = sc1%2
        sc1 = div(sc1,2)
        
        F -= graph[i,i]*qi  
        
        sc2 = state_code
        for j in 1:N
            qj = sc2%2
            sc2 = div(sc2,2)
                       
            low, high = i < j ? (i, j) : (j, i) 
            F -= graph[low,high]*qi*qj
        end
    end
    return F
end

function kernel(qubo, energies)
    threadsPerBlock = blockDim().x

    N = size(qubo)[1]

    i = blockIdx().x
    j = threadIdx().x

    state_code = (i - 1) * blockDim().x + j 

    F = _energy(state_code, qubo) |> Float32
    
    energies[state_code] = F
          
    return
end

function main()
    k = 2
    
    energies = CUDA.zeros(2^N) #CUDA.zeros(2*2^k)
    
    threadsPerBlock::Int64 = 2^k
    blocksPerGrid::Int64 = 2^(N-k)

    @cuda blocks=(blocksPerGrid) threads=(threadsPerBlock) kernel(cu_qubo, energies)
   
#     states = sortperm(energies)
#     energies[states]
    
    sort!(energies)
    
end

@btime begin
    main()
end

main()

  798.262 ms (81 allocations: 2.86 KiB)


16777216-element CuArray{Float32, 1}:
 -24231.07
 -24007.73
 -23786.467
 -23737.676
 -23559.938
 -23547.195
 -23515.354
 -23501.764
 -23352.525
 -23346.111
 -23314.717
 -23264.643
 -23252.385
      ⋮
    -80.06949
    -75.29317
    -54.983406
    -34.3692
    -24.935303
      0.0
     68.148834
    128.59712
    205.54321
    214.60916
    313.97363
    368.96417

In [11]:
function _energy(state_code, graph)
    F = 0
    N = size(graph)[1]
    
    sc1 = state_code    
    for i in 1:N
        qi = sc1%2
        sc1 = div(sc1,2)
        
        F -= graph[i,i]*qi  
        
        sc2 = state_code
        for j in 1:N
            qj = sc2%2
            sc2 = div(sc2,2)
                       
            low, high = i < j ? (i, j) : (j, i) 
            F -= graph[low,high]*qi*qj
        end
    end
    return F
end

function kernel2(qubo, energies, part_lst, part_st)
    threadsPerBlock = blockDim().x

    N = size(qubo)[1]

    i = blockIdx().x
    j = threadIdx().x

    state_code = (i - 1) * blockDim().x + j 

    F = _energy(state_code, qubo) |> Float32
    
    energies[state_code] = F
    
    sync_threads()
    
    if j == 1
        k = (i - 1) * blockDim().x + 1
        n = blockDim().x
        nr_of_block = gridDim().x
        
#         for ii in k:k+n-1
#             value = low_en[ii+1]
#             jj = ii
#             while jj > 0 && low_en[jj] > value
#                 low_en[jj+1] = low_en[jj]
#                 jj -= 1
#             end
#             low_en[i] = value
#         end 
        
        value=energies[k]
        st=k
        for ii in k:k+n-1
            if value > energies[ii]
                value = energies[ii]
                st=k
            end
        end 
               
        sync_threads()
        
        part_lst[i] = value # low_en[k]
        part_st[i] = st
    end
    
    return
end

function main()
    k = 2
    
    energies = CUDA.zeros(2^N) #CUDA.zeros(2*2^k)
    
    part_st = CUDA.zeros(2^(N-k))
    part_lst = CUDA.zeros(2^(N-k))
    
    threadsPerBlock::Int64 = 2^k
    blocksPerGrid::Int64 = 2^(N-k)

    @cuda blocks=(blocksPerGrid) threads=(threadsPerBlock) kernel2(cu_qubo, energies, part_lst, part_st)
   
#     states = sortperm(energies)
#     energies[states]
    
    sort!(part_lst)
    
end

@btime begin
    main()
end

  159.389 ms (107 allocations: 3.59 KiB)


4194304-element CuArray{Float32, 1}:
 -24231.07
 -24007.73
 -23786.467
 -23559.938
 -23547.195
 -23501.764
 -23352.525
 -23346.111
 -23314.717
 -23264.643
 -23252.385
 -23223.72
 -23221.188
      ⋮
  -1594.0449
  -1586.7893
  -1575.7623
  -1494.4044
  -1469.015
  -1441.9442
  -1417.1597
  -1309.0532
  -1231.5718
   -996.0316
   -961.8617
   -740.83167